In [ ]:

##클로드코드 폴더호출
#cd "/mnt/c/Users/redsk/OneDrive/デスクトップ/mikael_project"
#cd "/mnt/c/Users/redsk/OneDrive/デスクトップ/mikael_project/test_folder"

In [ ]:
################r 검증완료 #################
# =============================================================================
# 🚀 그룹 1: 통일된 함수명 - 마이리얼트립 크롤링 시스템 (리팩토링 완료)
# - 도시 정보를 UNIFIED_CITY_INFO로 통합하여 단일 소스로 관리
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    "LONGER_DELAYS": True,           # 새로 추가
    "MEMORY_CLEANUP_INTERVAL": 5,    # 새로 추가
    "MAX_PRODUCTS_PER_CITY": 2,     # 2 → 10개로 증가⭐⭐⭐⭐⭐⭐⭐⭐⭐
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["푸켓"]

# =============================================================================
# 📍 [최종 수정본] 단일 정보 소스 및 리팩토링된 함수
# =============================================================================

# 모든 도시의 상세 정보가 포함된 딕셔너리 (최종 확장본)
UNIFIED_CITY_INFO = {
    # 동남아시아
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK"},
    "아유타야": {"대륙": "아시아", "국가": "태국", "코드": "BKK"}, # 방콕 공항 사용
    "치앙마이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"},
    "빠이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"}, # 치앙마이 공항 사용
    "치앙라이": {"대륙": "아시아", "국가": "태국", "코드": "CEI"},
    "푸켓": {"대륙": "아시아", "국가": "태국", "코드": "HKT"},
    "피피": {"대륙": "아시아", "국가": "태국", "코드": "KBV"}, # 크라비 공항 사용
    "크라비": {"대륙": "아시아", "국가": "태국", "코드": "KBV"},
    "후아힌": {"대륙": "아시아", "국가": "태국", "코드": "HHQ"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG"},
    "쿠알라룸푸르": {"대륙": "아시아", "국가": "말레이시아", "코드": "KUL"},
    "코타키나발루": {"대륙": "아시아", "국가": "말레이시아", "코드": "BKI"},
    "페낭": {"대륙": "아시아", "국가": "말레이시아", "코드": "PEN"},
    "랑카위": {"대륙": "아시아", "국가": "말레이시아", "코드": "LGK"},
    "세부": {"대륙": "아시아", "국가": "필리핀", "코드": "CEB"},
    "보홀": {"대륙": "아시아", "국가": "필리핀", "코드": "TAG"},
    "마닐라": {"대륙": "아시아", "국가": "필리핀", "코드": "MNL"},
    "보라카이": {"대륙": "아시아", "국가": "필리핀", "코드": "MPH"}, # 카티클란 공항
    "다낭": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"},
    "호이안": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"}, # 다낭 공항 사용
    "후에": {"대륙": "아시아", "국가": "베트남", "코드": "HUI"},
    "호치민": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"},
    "무이네": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"}, # 호치민 공항 사용
    "푸꾸옥": {"대륙": "아시아", "국가": "베트남", "코드": "PQC"},
    "나트랑": {"대륙": "아시아", "국가": "베트남", "코드": "CXR"},
    "하노이": {"대륙": "아시아", "국가": "베트남", "코드": "HAN"},
    "달랏": {"대륙": "아시아", "국가": "베트남", "코드": "DLI"},
    "발리": {"대륙": "아시아", "국가": "인도네시아", "코드": "DPS"},
    "프놈펜": {"대륙": "아시아", "국가": "캄보디아", "코드": "PNH"}, #아직 상품없음 25.0723
    "시엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, #아직 상품없음 25.0723
    "씨엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, # 시엠립 동의어, #아직 상품없음 25.0723
    "비엔티안": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"},
    "방비엥": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"}, # 비엔티안 공항 사용
    "루앙프라방": {"대륙": "아시아", "국가": "라오스", "코드": "LPQ"},

    # 동북아시아
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX"},
    "교토": {"대륙": "아시아", "국가": "일본", "코드": "KIX"}, # 오사카 공항 사용
    "나고야": {"대륙": "아시아", "국가": "일본", "코드": "NGO"},
    "후쿠오카": {"대륙": "아시아", "국가": "일본", "코드": "FUK"},
    "벳푸": {"대륙": "아시아", "국가": "일본", "코드": "OIT"}, # 오이타 공항 사용
    "오이타": {"대륙": "아시아", "국가": "일본", "코드": "OIT"},
    "구마모토": {"대륙": "아시아", "국가": "일본", "코드": "KMJ"},
    "오키나와": {"대륙": "아시아", "국가": "일본", "코드": "OKA"},
    "미야코지마": {"대륙": "아시아", "국가": "일본", "코드": "MMY"},
    "삿포로": {"대륙": "아시아", "국가": "일본", "코드": "CTS"},
    "타이베이": {"대륙": "아시아", "국가": "대만", "코드": "TPE"},
    "상하이": {"대륙": "아시아", "국가": "중국", "코드": "PVG"},
    "베이징": {"대륙": "아시아", "국가": "중국", "코드": "PEK"},
    "하이난(싼야)": {"대륙": "아시아", "국가": "중국", "코드": "SYX"},
    "마카오": {"대륙": "아시아", "국가": "마카오", "코드": "MFM"},
    
    # 남아시아
    "뉴델리": {"대륙": "아시아", "국가": "인도", "코드": "DEL"},
    "뭄바이": {"대륙": "아시아", "국가": "인도", "코드": "BOM"},
    "카트만두": {"대륙": "아시아", "국가": "네팔", "코드": "KTM"},
    "포카라": {"대륙": "아시아", "국가": "네팔", "코드": "PKR"},

    # 한국
    "서울": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "부산": {"대륙": "아시아", "국가": "대한민국", "코드": "PUS"},
    "제주": {"대륙": "아시아", "국가": "대한민국", "코드": "CJU"},
    "대구": {"대륙": "아시아", "국가": "대한민국", "코드": "TAE"},
    "광주": {"대륙": "아시아", "국가": "대한민국", "코드": "KWJ"},
    "여수": {"대륙": "아시아", "국가": "대한민국", "코드": "RSU"},
    "인천": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "김포": {"대륙": "아시아", "국가": "대한민국", "코드": "GMP"},

    # 유럽
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG"},
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR"},
    "더블린": {"대륙": "유럽", "국가": "아일랜드", "코드": "DUB"},
    "로마": {"대륙": "유럽", "국가": "이탈리아", "코드": "FCO"},
    "피렌체": {"대륙": "유럽", "국가": "이탈리아", "코드": "FLR"},
    "베네치아": {"대륙": "유럽", "국가": "이탈리아", "코드": "VCE"},
    "밀라노": {"대륙": "유럽", "국가": "이탈리아", "코드": "MXP"},
    "바르셀로나": {"대륙": "유럽", "국가": "스페인", "코드": "BCN"},
    "마드리드": {"대륙": "유럽", "국가": "스페인", "코드": "MAD"},
    "세비야": {"대륙": "유럽", "국가": "스페인", "코드": "SVQ"},
    "그라나다": {"대륙": "유럽", "국가": "스페인", "코드": "GRX"},
    "이비자": {"대륙": "유럽", "국가": "스페인", "코드": "IBZ"},
    "리스본": {"대륙": "유럽", "국가": "포르투갈", "코드": "LIS"},
    "포르투": {"대륙": "유럽", "국가": "포르투갈", "코드": "OPO"},
    "프라하": {"대륙": "유럽", "국가": "체코", "코드": "PRG"},
    "비엔나": {"대륙": "유럽", "국가": "오스트리아", "코드": "VIE"},
    "취리히": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"},
    "인터라켄": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"}, # 취리히 공항 사용
    "암스테르담": {"대륙": "유럽", "국가": "네덜란드", "코드": "AMS"},
    "브뤼셀": {"대륙": "유럽", "국가": "벨기에", "코드": "BRU"},
    "뮌헨": {"대륙": "유럽", "국가": "독일", "코드": "MUC"},
    "베를린": {"대륙": "유럽", "국가": "독일", "코드": "BER"},
    "프랑크푸르트": {"대륙": "유럽", "국가": "독일", "코드": "FRA"},
    "부다페스트": {"대륙": "유럽", "국가": "헝가리", "코드": "BUD"},
    "바르샤바": {"대륙": "유럽", "국가": "폴란드", "코드": "WAW"},
    "크라쿠프": {"대륙": "유럽", "국가": "폴란드", "코드": "KRK"},
    "아테네": {"대륙": "유럽", "국가": "그리스", "코드": "ATH"},
    "산토리니": {"대륙": "유럽", "국가": "그리스", "코드": "JTR"},
    "자그레브": {"대륙": "유럽", "국가": "크로아티아", "코드": "ZAG"},
    "두브로브니크": {"대륙": "유럽", "국가": "크로아티아", "코드": "DBV"},
    "코펜하겐": {"대륙": "유럽", "국가": "덴마크", "코드": "CPH"},
    "스톡홀름": {"대륙": "유럽", "국가": "스웨덴", "코드": "ARN"},
    "오슬로": {"대륙": "유럽", "국가": "노르웨이", "코드": "OSL"},
    "헬싱키": {"대륙": "유럽", "국가": "핀란드", "코드": "HEL"},
    "이스탄불": {"대륙": "유럽", "국가": "터키", "코드": "IST"},

    # 북미
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국", "코드": "LAX"},
    "시카고": {"대륙": "북미", "국가": "미국", "코드": "ORD"},
    "하와이": {"대륙": "북미", "국가": "미국", "코드": "HNL"},
    "샌프란시스코": {"대륙": "북미", "국가": "미국", "코드": "SFO"},
    "라스베이거스": {"대륙": "북미", "국가": "미국", "코드": "LAS"},
    "워싱턴 D.C.": {"대륙": "북미", "국가": "미국", "코드": "IAD"},
    "보스턴": {"대륙": "북미", "국가": "미국", "코드": "BOS"},
    "시애틀": {"대륙": "북미", "국가": "미국", "코드": "SEA"},
    "밴쿠버": {"대륙": "북미", "국가": "캐나다", "코드": "YVR"},
    "토론토": {"대륙": "북미", "국가": "캐나다", "코드": "YYZ"},
    "칸쿤": {"대륙": "북미", "국가": "멕시코", "코드": "CUN"},

    # 오세아니아
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD"},
    "멜버른": {"대륙": "오세아니아", "국가": "호주", "코드": "MEL"},
    "괌": {"대륙": "오세아니아", "국가": "괌", "코드": "GUM"},
    "사이판": {"대륙": "오세아니아", "국가": "북마리아나 제도", "코드": "SPN"},
    "오클랜드": {"대륙": "오세아니아", "국가": "뉴질랜드", "코드": "AKL"},
    
    # 중동
    "두바이": {"대륙": "아시아", "국가": "아랍에미리트", "코드": "DXB"},
}



print(f"✅ UNIFIED_CITY_INFO 로드 완료! {len(UNIFIED_CITY_INFO)}개 도시 지원")

# =============================================================================
# 🔧 핵심 함수들 - 단일 정보 소스(UNIFIED_CITY_INFO) 사용
# =============================================================================

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (UNIFIED_CITY_INFO 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        code = info.get("코드", city_name[:3].upper())
        return code
    return city_name[:3].upper()

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (사전 정의된 값만 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 정의되지 않은 도시에 대한 기본값
        return "기타", "기타"

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (기존: get_product_name_by_type → 새로운: get_product_name)"""
    print(f"  📊 {url_type} 상품명 수집 중...")
    
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """✅ 가격 수집 - 수정된 버전 (잘못된 텍스트 필터링 강화)"""
    print(f"  💰 가격 정보 수집 중...")
    
    # 마이리얼트립 실제 가격 셀렉터들
    price_selectors = [
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 87, 87)']"),
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 71, 71)']"),
        (By.CSS_SELECTOR, "span[style*='color: red']"),
        (By.CSS_SELECTOR, ".text-red"),
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        # 더 구체적인 XPath
        (By.XPATH, "//span[contains(text(), '원~') and contains(text(), ',') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), '원-') and contains(text(), ',') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), ',') and contains(text(), '원') and not(contains(text(), '인원')) and not(contains(text(), '최소'))]"),
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_elements = driver.find_elements(selector_type, selector_value)
            
            for price_element in price_elements:
                found_price = price_element.text.strip()
                
                if not found_price:
                    continue
                
                # 🔥 강력한 필터링: 가격이 아닌 텍스트들 제외
                invalid_keywords = [
                    '쿠폰', '받기', '다운', '할인', '적립', '포인트',
                    '최소', '인원', '명', '최대', '선택', '옵션',
                    '예약', '신청', '문의', '상담', '확인', '명부터',
                    '시간', '일정', '코스', '투어', '여행', '부터',
                    '언어', '가이드', '포함', '불포함', '이상',
                    '취소', '환불', '변경', '안내', '모집'
                ]
                
                if any(keyword in found_price for keyword in invalid_keywords):
                    continue
                
                # 길이 제한 (너무 긴 텍스트 제외)
                if len(found_price) > 30:
                    continue
                
                # 가격 패턴 확인
                import re
                price_patterns = [
                    r'\d{1,3}(?:,\d{3})+원[~-]?',  # 10,000원~ 형태
                    r'\d+,\d+원[~-]?',             # 간단한 천단위 구분
                    r'\d{4,}원[~-]?',              # 10000원~ 형태
                ]
                
                is_valid_price = any(re.search(pattern, found_price) for pattern in price_patterns)
                
                if is_valid_price and '원' in found_price:
                    print(f"    ✅ 유효한 가격 발견: '{found_price}'")
                    return found_price
                    
        except Exception:
            continue
    
    print(f"    ❌ 가격 정보를 찾을 수 없습니다")
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (기존: extract_clean_price → 새로운: clean_price) (공용 함수)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    price_pattern = r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?'
    match = re.search(price_pattern, price_text)
    
    if match:
        return match.group(1) + "원"
    else:
        return price_text

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text

def download_image(driver, product_name, city_name, product_index):
    """✅ 이미지 다운로드 - 계층 구조 저장 (대륙/국가/도시/)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': '',
            'path': '',
            'relative_path': '',
            'size': 0
        }
        
    print(f"  🖼️ 대표 상품 이미지 다운로드 중...")
    
    # 더 광범위한 이미지 셀렉터들
    image_selectors = [
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.CSS_SELECTOR, ".image-gallery img:first-child"),
        (By.CSS_SELECTOR, ".slider img:first-child"),
        (By.XPATH, "//img[@width and @height and (@width > '200' or @height > '200')]"),
        (By.XPATH, "//img[contains(@src, 'large') or contains(@src, 'big') or contains(@src, 'main')]"),
        (By.CSS_SELECTOR, "img[src*='cdn']"),
        (By.CSS_SELECTOR, "img[src*='cloudfront']"),
        (By.CSS_SELECTOR, "img[src*='amazonaws']"),
        (By.XPATH, "(//img[contains(@src, 'http') and not(contains(@src, 'icon')) and not(contains(@src, 'logo'))])[1]"),
        (By.XPATH, "//img[contains(@alt, '상품') or contains(@alt, '투어') or contains(@alt, '여행')]"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            for img_element in img_elements:
                try:
                    img_url = img_element.get_attribute('src')
                    if not img_url or not img_url.startswith('http'):
                        continue
                    exclude_patterns = ['logo', 'icon', 'banner', 'ad', 'avatar', 'profile', 'button', 'arrow', 'star', 'thumb', 'small', 'mini']
                    if any(pattern in img_url.lower() for pattern in exclude_patterns):
                        continue
                    try:
                        size = img_element.size
                        if size and (size.get('width', 0) < 100 or size.get('height', 0) < 100):
                            continue
                    except:
                        pass
                    print(f"    ✅ 이미지 URL 발견: {img_url[:50]}...")
                    break
                except Exception:
                    continue
            if img_url:
                break
        except Exception:
            continue

    if not img_url:
        print(f"    ❌ 이미지 URL을 찾을 수 없습니다")
        return {
            'status': '이미지 URL 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'relative_path': '',
            'size': 0
        }

    # 🏗️ 계층 구조 폴더 생성 및 이미지 다운로드
    try:
        import urllib.request
        import os
        
        # 도시 정보 가져오기
        continent, country = get_city_info(city_name)
        city_code = get_city_code(city_name)
        img_filename = f"{city_code}_{product_index:03d}.jpg"
        
        # 📁 계층 구조 폴더 경로 생성: myrealtripthumb_img/대륙/국가/도시/
        base_folder = "myrealtripthumb_img"
        hierarchical_folder = os.path.join(base_folder, continent, country, city_name)
        
        # 폴더 생성 (계층 구조)
        os.makedirs(hierarchical_folder, exist_ok=True)
        print(f"    📁 계층 폴더 생성: {hierarchical_folder}")
        
        # 전체 파일 경로
        img_path = os.path.join(hierarchical_folder, img_filename)
        
        # 상대 경로 (CSV에 저장용)
        relative_path = os.path.join(continent, country, city_name, img_filename)
        
        # User-Agent 추가로 다운로드 성공률 높이기
        req = urllib.request.Request(
            img_url,
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
        )
        
        with urllib.request.urlopen(req, timeout=10) as response:
            with open(img_path, 'wb') as f:
                f.write(response.read())
        
        file_size = os.path.getsize(img_path)
        
        if file_size < 1024:
            os.remove(img_path)
            print(f"    ❌ 이미지 파일이 너무 작습니다 ({file_size} bytes)")
            return {
                'status': '파일 너무 작음',
                'filename': img_filename,
                'path': '',
                'relative_path': '',
                'size': 0
            }
        
        print(f"    ✅ 계층 구조 이미지 저장 완료! ({file_size:,} bytes)")
        print(f"    📍 저장 위치: {relative_path}")
        
        return {
            'status': '다운로드 완료',
            'filename': img_filename,
            'path': img_path,
            'relative_path': relative_path,
            'size': file_size
        }
        
    except Exception as e:
        print(f"    ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
        return {
            'status': f'다운로드 실패: {type(e).__name__}',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'relative_path': '',
            'size': 0
        }
    else:
        return {
            'status': '이미지 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'size': 0
        }

def save_results(products_data):
    """✅ 데이터 저장 (기존: save_myrealtrip_data → 새로운: save_results)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    df = pd.DataFrame(products_data)
    csv_path = f"myrealtrip_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📁 개별 CSV 저장 완료: {csv_path}")
    
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "city": city_name
        }
    }
    
    try:
        with open('data_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        print(f"📁 메타데이터 저장 완료: data_metadata.json")
    except Exception as e:
        print(f"⚠️ 메타데이터 저장 실패: {e}")
    
    return csv_path

# =============================================================================
# 🔄 1순위: 상태 관리 시스템 구축 (데이터 연속성 확보)
# =============================================================================

def load_crawler_state():
    """크롤링 상태 로드 (config/crawler_meta.json에서)"""
    config_dir = "config"
    state_file = os.path.join(config_dir, "crawler_meta.json")
    urls_file = os.path.join(config_dir, "completed_urls.log")
    
    # config 폴더 생성
    os.makedirs(config_dir, exist_ok=True)
    
    # 기본 상태
    default_state = {
        "total_collected_count": 0,
        "last_crawled_page": 1,
        "current_session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    # 상태 파일 로드
    if os.path.exists(state_file):
        try:
            with open(state_file, 'r', encoding='utf-8') as f:
                state = json.load(f)
            print(f"✅ 상태 파일 로드: {state['total_collected_count']}개 수집 완료")
        except Exception as e:
            print(f"⚠️ 상태 파일 로드 실패: {e}, 기본값 사용")
            state = default_state
    else:
        state = default_state
        print("🆕 새로운 크롤링 세션 시작")
    
    # 완료된 URL 목록 로드
    completed_urls = set()
    if os.path.exists(urls_file):
        try:
            with open(urls_file, 'r', encoding='utf-8') as f:
                completed_urls = set(line.strip() for line in f if line.strip())
            print(f"✅ 완료된 URL {len(completed_urls)}개 로드")
        except Exception as e:
            print(f"⚠️ URL 파일 로드 실패: {e}")
    
    return state, completed_urls

def save_crawler_state(state, new_url=None):
    """크롤링 상태 저장 (즉시 저장)"""
    config_dir = "config"
    state_file = os.path.join(config_dir, "crawler_meta.json")
    urls_file = os.path.join(config_dir, "completed_urls.log")
    
    # 상태 업데이트
    state["last_updated"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    try:
        # 상태 파일 저장
        with open(state_file, 'w', encoding='utf-8') as f:
            json.dump(state, f, ensure_ascii=False, indent=2)
        
        # 새 URL이 있으면 추가
        if new_url:
            with open(urls_file, 'a', encoding='utf-8') as f:
                f.write(new_url + '\n')
            state["total_collected_count"] += 1
        
        return True
    except Exception as e:
        print(f"❌ 상태 저장 실패: {e}")
        return False

def save_batch_data(batch_results, city_name):
    """배치 데이터 저장 (I/O 효율성 증대)"""
    if not batch_results:
        return None
    
    try:
        # 도시 정보 가져오기
        continent, country = get_city_info(city_name)
        
        # 계층 구조 폴더 생성: data/대륙/국가/도시/
        data_dir = os.path.join("data", continent, country, city_name)
        os.makedirs(data_dir, exist_ok=True)
        
        # 파일명 설정
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # 1. 도시별 CSV (덮어쓰기 방식)
        city_csv = os.path.join(data_dir, f"myrealtrip_{city_name}_products.csv")
        df = pd.DataFrame(batch_results)
        df.to_csv(city_csv, index=False, encoding='utf-8-sig')
        
        # 2. 국가별 통합 CSV (추가 방식)
        country_dir = os.path.join("data", continent, country)
        country_csv = os.path.join(country_dir, f"{country}_myrealtrip_products_all.csv")
        
        # 기존 파일이 있으면 추가, 없으면 새로 생성
        if os.path.exists(country_csv):
            df.to_csv(country_csv, mode='a', header=False, index=False, encoding='utf-8-sig')
        else:
            df.to_csv(country_csv, index=False, encoding='utf-8-sig')
        
        print(f"✅ 배치 데이터 저장 완료:")
        print(f"   📁 도시별: {city_csv}")
        print(f"   📁 국가별: {country_csv}")
        
        return {
            "city_csv": city_csv,
            "country_csv": country_csv,
            "data_count": len(batch_results)
        }
        
    except Exception as e:
        print(f"❌ 배치 데이터 저장 실패: {e}")
        return None

def filter_new_urls(all_urls, completed_urls):
    """완료되지 않은 새로운 URL만 필터링"""
    new_urls = [url for url in all_urls if url not in completed_urls]
    
    print(f"🔍 URL 필터링 결과:")
    print(f"   📊 전체 URL: {len(all_urls)}개")
    print(f"   ✅ 완료된 URL: {len(completed_urls)}개")
    print(f"   🆕 새로운 URL: {len(new_urls)}개")
    
    return new_urls

def create_image_filename(total_count, city_name):
    """총 수집 개수 기준으로 이미지 파일명 생성"""
    city_code = get_city_code(city_name)
    return f"{city_code}_{total_count:03d}.jpg"

# =============================================================================
# 🚀 Phase 2: 확장성 개선 시스템 (리팩토링된 버전)
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장 (UNIFIED_CITY_INFO 기반)"""
    enhanced_city_data = {
        "version": "3.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(UNIFIED_CITY_INFO)
    }

    for city_name, info in UNIFIED_CITY_INFO.items():
        enhanced_city_data["cities"][city_name] = {
            "code": info.get("코드", "N/A"),
            "continent": info.get("대륙", "기타"),
            "country": info.get("국가", "기타")
        }
    
    try:
        with open('city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ city_codes.json 파일 생성 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드 (UNIFIED_CITY_INFO와 동기화)"""
    if not os.path.exists('city_codes.json'):
        print("📝 city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
    
    try:
        with open('city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        loaded_cities = city_data.get("cities", {})
        
        for city, info in loaded_cities.items():
            if city not in UNIFIED_CITY_INFO:
                 UNIFIED_CITY_INFO[city] = {
                     "대륙": info.get("continent"),
                     "국가": info.get("country"),
                     "코드": info.get("code")
                 }
        
        print(f"✅ city_codes.json 로드 및 동기화 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 코드의 UNIFIED_CITY_INFO를 사용합니다.")

def add_new_city(city_name, airport_code, continent="기타", country="기타", update_file=True):
    """새로운 도시를 UNIFIED_CITY_INFO에 추가하는 함수"""
    global UNIFIED_CITY_INFO
    
    if city_name not in UNIFIED_CITY_INFO:
        UNIFIED_CITY_INFO[city_name] = {
            "대륙": continent,
            "국가": country,
            "코드": airport_code
        }
        print(f"✅ 메모리에 추가: {city_name} → {airport_code} ({continent}, {country})")
        if update_file:
            create_city_codes_file()
        return True
    else:
        print(f"이미 존재하는 도시입니다: {city_name}")
        return False

def show_supported_cities():
    """지원하는 도시 목록 표시 (UNIFIED_CITY_INFO 기반)"""
    print("\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    cities_by_continent = {}
    for city, info in UNIFIED_CITY_INFO.items():
        continent = info.get("대륙", "기타")
        if continent not in cities_by_continent:
            cities_by_continent[continent] = []
        cities_by_continent[continent].append(city)

    for continent, cities in sorted(cities_by_continent.items()):
        print(f"\n📍 {continent}:")
        for city in sorted(cities):
            code = UNIFIED_CITY_INFO[city].get("코드", "N/A")
            print(f"   {city} → {code}")
    
    print(f"\n📊 총 {len(UNIFIED_CITY_INFO)}개 도시 지원")
    print("="*50)

def validate_city(city_name):
    """도시명 유효성 검사 (UNIFIED_CITY_INFO 기반)"""
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in UNIFIED_CITY_INFO:
        code = UNIFIED_CITY_INFO[city_name].get("코드", "N/A")
        return True, f"지원하는 도시입니다. ({code})"
    
    similar_cities = [c for c in UNIFIED_CITY_INFO if city_name.lower() in c.lower() or c.lower() in city_name.lower()]
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    global CONFIG
    
    scalability_config = {
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def initialize_file_system():
    """파일 시스템 초기화 및 설정 (리팩토링된 버전)"""
    print("🔧 Phase 2: 확장성 개선 시스템 초기화...")
    
    update_config_for_scalability()
    
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        load_city_codes_from_file()
    
    print("✅ Phase 2 시스템 초기화 완료!")
    return True

# =============================================================================
# 🛠️ 기존 유틸리티 함수들 (Phase 2 시스템과 함께 유지)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭 (안정성 강화)"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            driver.execute_script("arguments[0].click();", view_all_button)
            
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
        
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print("  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def get_rating(driver):
    """평점 정보 수집 (기존 함수 유지)"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집 - 수정된 버전 (후기 내용 제외)"""
    print(f"  🌐 언어 정보 찾는 중...")
    
    # 언어 정보가 있을 가능성이 높은 셀렉터들
    language_selectors = [
        # 언어 섹션 직접 타겟팅
        (By.XPATH, "//dt[contains(text(), '언어')]/following-sibling::dd"),
        (By.XPATH, "//span[contains(@class, 'language')]"),
        (By.XPATH, "//div[contains(@class, 'language')]//span"),
        # 짧은 언어 텍스트만 (후기 제외)
        (By.XPATH, "//span[contains(text(), '한국어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '영어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '중국어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '일본어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), 'Korean') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), 'English') and string-length(text()) < 20]"),
        # 언어 아이콘 근처의 텍스트
        (By.XPATH, "//i[contains(@class, 'language')]/following-sibling::span"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_elements = driver.find_elements(selector_type, selector_value)
            
            for language_element in language_elements:
                language_text = language_element.text.strip()
                
                if not language_text:
                    continue
                
                # 🔥 강력한 필터링: 후기나 긴 텍스트 제외
                # 길이 제한 (언어 정보는 보통 짧음)
                if len(language_text) > 50:
                    continue
                
                # 후기 관련 키워드 제외
                review_keywords = [
                    '후기', '리뷰', '평가', '별점', '추천', '만족',
                    '여행', '투어', '경험', '좋았', '나빴', '최고',
                    '다음에', '또', '재방문', '친절', '서비스',
                    '가격', '시간', '일정', '코스', '가이드',
                    '예약', '신청', '문의', '확인', '취소'
                ]
                
                if any(keyword in language_text for keyword in review_keywords):
                    continue
                
                # 언어 키워드 확인
                language_keywords = [
                    '언어', '한국어', '영어', '중국어', '일본어', 
                    'Korean', 'English', 'Chinese', 'Japanese',
                    '가능', '지원', '제공'
                ]
                
                if any(keyword in language_text for keyword in language_keywords):
                    print(f"    ✅ 언어 정보 발견: {language_text}")
                    return language_text
                    
        except Exception:
            continue

    print(f"    ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

# =============================================================================
# Phase 2 시스템 자동 실행
# =============================================================================

try:
    initialize_file_system()
    
    # quick_add_cities()는 모든 도시가 UNIFIED_CITY_INFO에 포함되어 더 이상 필요 없음
    
    show_supported_cities()
    
    print("\n🎉 Phase 2: 확장성 개선 완료!")
    print("💡 이제 이런 기능들을 사용할 수 있습니다:")
    print("   - add_new_city('제주도', 'CJU', '아시아', '대한민국')  # 새 도시 추가")
    print("   - show_supported_cities()        # 지원 도시 목록")
    print("   - validate_city('방콕')          # 도시 유효성 검사")
    
except Exception as e:
    print(f"❌ Phase 2 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

print("\n" + "="*60)
print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print("="*60)
print("🔧 핵심 함수명 변경 완료:")
print("   get_product_name_by_type() → get_product_name()")
print("   get_price_fixed() → get_price()")
print("   download_image_improved_fixed() → download_image()")
print("   extract_clean_price() → clean_price()")
print("   extract_clean_rating() → clean_rating()")
print("   save_myrealtrip_data() → save_results()")
print("="*60)
print("🔄 1순위: 상태 관리 시스템 추가 완료!")
print("   - load_crawler_state()     # 크롤링 상태 로드")
print("   - save_crawler_state()     # 크롤링 상태 저장")
print("   - save_batch_data()        # 배치 데이터 저장")
print("   - filter_new_urls()        # 새로운 URL 필터링")
print("   - create_image_filename()  # 이미지 파일명 생성")
print("="*60)
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2,3,4에서 함수 호출을 통일된 함수명으로 변경하세요!")
print("🚨 Gemini 지적사항 모두 해결 완료! 안전하게 실행 가능합니다!")

In [32]:
# 🚀 그룹 2: 도시 하드코딩 해결 - 드라이버 실행 + 도시 검색
# (이 셀을 실행하면 브라우저가 열리고 그룹 1에서 설정한 도시 검색까지 완료됩니다)

print("🔄 마이리얼트립 크롤링 시작!")

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 크롬 드라이버 실행
try:
    driver = setup_driver()
except Exception as e:
    print(f"❌ 드라이버 설정 실패: {e}")
    print("⚠️ 그룹 1을 먼저 실행했는지 확인하세요!")

# 썸네일 폴더 생성
if CONFIG["SAVE_IMAGES"]:
    img_folder_path = os.path.abspath("") + "/myrealtripthumb_img"
    try:
        shutil.rmtree(img_folder_path)
    except:
        pass
    os.makedirs(img_folder_path, exist_ok=True)
    print("📁 이미지 폴더 생성 완료")

# 🆕 그룹 1 설정에서 도시 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

print("="*60)
print(f"🌏 그룹 1 설정 도시: {city_name} 검색 시작!")
print(f"⚙️  설정: 재시도 {CONFIG['RETRY_COUNT']}회, 타임아웃 {CONFIG['WAIT_TIMEOUT']}초")
print("="*60)

print(f"🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)")
print(f"📍 현재 작업: {city_name} - 진행중")
print("-" * 50)
print(f"  🌍 대륙: {continent} | 국가: {country}")

try:
    # 1. 메인 페이지로 이동
    print("  📱 마이리얼트립 메인 페이지 이동 중...")
    retry_operation(
        lambda: go_to_main_page(driver), 
        "메인 페이지 이동"
    )
    print(f"  ✅ 마이리얼트립 페이지 열기 완료")
    
    # 2. 검색창 찾기 및 입력 (🆕 동적 도시명 사용)
    retry_operation(
        lambda: find_and_fill_search(driver, city_name), 
        f"{city_name} 검색창 입력"
    )

    # 3. 검색 버튼 클릭
    retry_operation(
        lambda: click_search_button(driver), 
        "검색 버튼 클릭"
    )

    # 4. 팝업 처리 (선택사항)
    try:
        handle_popup(driver)
    except Exception as e:
        print(f"  ℹ️ 팝업 처리 중 오류 (무시됨): {type(e).__name__}")

    # 5. 전체 상품 보기 클릭 (선택사항)
    try:
        click_view_all(driver)
    except Exception as e:
        print(f"  ℹ️ 전체 상품 보기 처리 중 오류 (무시됨): {type(e).__name__}")

    print(f"\n✅ 그룹 2 완료: {city_name} 검색 성공!")
    print(f"🎯 현재 상태: {city_name} 상품 목록 페이지에 있습니다")
    print("🚀 다음: 그룹 3을 실행하여 URL 수집 테스트하세요!")
    
except TimeoutException as e:
    print(f"  ⏰ {city_name}: 페이지 로딩 시간 초과")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 시간 초과")
    
except NoSuchElementException as e:
    print(f"  🔍 {city_name}: 필요한 웹 요소를 찾을 수 없음")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 요소 없음")
    
except Exception as e:
    print(f"  ❌ {city_name}: 예상치 못한 오류 - {type(e).__name__}: {e}")
    print("❌ 그룹 2 실패: 예상치 못한 오류")
    print("💡 그룹 1을 먼저 실행했는지 확인하세요!")
    print(f"💡 CITIES_TO_SEARCH에 '{city_name}'이 올바르게 설정되었는지 확인하세요!")
    

🔄 마이리얼트립 크롤링 시작!
🔄 결과 저장소 초기화 완료
✅ 크롬 드라이버 실행 성공!
Windows
📁 이미지 폴더 생성 완료
🌏 그룹 1 설정 도시: 푸켓 검색 시작!
⚙️  설정: 재시도 3회, 타임아웃 10초
🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)
📍 현재 작업: 푸켓 - 진행중
--------------------------------------------------
  🌍 대륙: 아시아 | 국가: 태국
  📱 마이리얼트립 메인 페이지 이동 중...
  ✅ 마이리얼트립 페이지 열기 완료
  🔍 '푸켓' 검색창 찾는 중...
  ✅ 검색창을 찾았습니다!
  📝 '푸켓' 키워드 입력 완료
  🔎 검색 버튼 찾는 중...
  ✅ 검색 버튼 클릭 성공!
  ✅ 팝업창을 닫았습니다.
  📋 전체 상품 보기 버튼 찾는 중...
  ✅ 전체 상품 보기 클릭 성공!

✅ 그룹 2 완료: 푸켓 검색 성공!
🎯 현재 상태: 푸켓 상품 목록 페이지에 있습니다
🚀 다음: 그룹 3을 실행하여 URL 수집 테스트하세요!


In [33]:
# 🔍 그룹 3: URL 수집 테스트 (24개 완전 수집 버전)
# (이 셀을 실행하면 현재 페이지에서 Products + Offers 24개 URL을 모두 수집합니다)

print("🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)")
print("="*50)

def collect_all_24_urls(driver):
    print(f"  🔍 Products와 Offers URL 모두 수집 중...")
    
    # 페이지 로딩 완료 대기
    time.sleep(5)
    
    all_links = driver.find_elements(By.TAG_NAME, "a")
    print(f"  📊 전체 링크 수: {len(all_links)}개")
    
    products_urls = []
    offers_urls = []
    
    for link in all_links:
        try:
            href = link.get_attribute('href')
            if href:
                if 'experiences.myrealtrip.com/products/' in href:
                    if href not in products_urls:
                        products_urls.append(href)
                elif 'myrealtrip.com/offers/' in href:
                    if href not in offers_urls:
                        offers_urls.append(href)
        except:
            continue
    
    # 두 리스트 합치기 (products 먼저, offers 나중에)
    all_urls = products_urls + offers_urls
    
    print(f"  ✅ Products URL: {len(products_urls)}개")
    print(f"  ✅ Offers URL: {len(offers_urls)}개")
    print(f"  🎉 총 수집: {len(all_urls)}개")
    
    return all_urls

try:
    # 24개 URL 수집 실행
    print("📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...")
    
    collected_urls = collect_all_24_urls(driver)
    
    if collected_urls:
        print(f"\n🎉 URL 수집 성공!")
        print(f"📈 총 {len(collected_urls)}개의 상품/오퍼 URL을 찾았습니다!")
        
        # 설정된 개수와 비교
        max_products = CONFIG['MAX_PRODUCTS_PER_CITY']
        will_crawl = min(len(collected_urls), max_products)
        
        print(f"🔢 설정된 크롤링 개수: {max_products}개")
        print(f"🎯 실제 크롤링할 개수: {will_crawl}개")
        
        print(f"\n📋 수집된 URL 목록:")
        print("-" * 80)
        for i, url in enumerate(collected_urls, 1):
            url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
            print(f"  {i:2d}. {url_type}: {url}")
        
        # URL 유효성 체크
        print(f"\n🔍 URL 유효성 체크:")
        products_count = sum(1 for url in collected_urls if '/products/' in url)
        offers_count = sum(1 for url in collected_urls if '/offers/' in url)
        
        print(f"  ✅ Products: {products_count}개")
        print(f"  ✅ Offers: {offers_count}개")
        print(f"  🎯 총 유효 URL: {products_count + offers_count}개")
        
        if len(collected_urls) >= 20:
            print("\n🚀 URL 수집 대성공! 다음 단계로 진행 가능합니다!")
            print("✅ 그룹 3 완료: 24개 URL 수집 테스트 성공")
            print("🎯 다음: 그룹 4를 실행하여 24개 상품/오퍼 크롤링을 시작하세요!")
        else:
            print("\n⚠️ 수집된 URL이 예상보다 적습니다.")
            
    else:
        print("❌ URL 수집 실패: 상품/오퍼 URL을 찾을 수 없습니다")
        print("💡 추가 디버깅:")
        
        # 디버깅 정보
        try:
            page_source = driver.page_source
            products_count = page_source.count('experiences.myrealtrip.com/products/')
            offers_count = page_source.count('myrealtrip.com/offers/')
            print(f"   📄 페이지 소스에서 products 패턴: {products_count}회")
            print(f"   📄 페이지 소스에서 offers 패턴: {offers_count}회")
            
        except Exception as debug_error:
            print(f"   ❌ 디버깅 실패: {debug_error}")
        
        print("🔄 그룹 2를 다시 실행해보세요!")
        
except NameError as e:
    print(f"❌ 변수 오류: {e}")
    print("💡 그룹 1과 그룹 2를 먼저 실행했는지 확인하세요!")
    
except Exception as e:
    print(f"❌ URL 수집 중 오류: {type(e).__name__}: {e}")
    print("💡 가능한 해결책:")
    print("   1. 그룹 2가 성공적으로 완료되었는지 확인")
    print("   2. 브라우저가 방콕 상품 목록 페이지에 있는지 확인")
    print("   3. 인터넷 연결 상태 확인")

print("\n" + "="*50)

🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)
📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...
  🔍 Products와 Offers URL 모두 수집 중...
  📊 전체 링크 수: 52개
  ✅ Products URL: 17개
  ✅ Offers URL: 7개
  🎉 총 수집: 24개

🎉 URL 수집 성공!
📈 총 24개의 상품/오퍼 URL을 찾았습니다!
🔢 설정된 크롤링 개수: 2개
🎯 실제 크롤링할 개수: 2개

📋 수집된 URL 목록:
--------------------------------------------------------------------------------
   1. 🛍️ Product: https://experiences.myrealtrip.com/products/3446452
   2. 🛍️ Product: https://experiences.myrealtrip.com/products/3446456
   3. 🛍️ Product: https://experiences.myrealtrip.com/products/3147846
   4. 🛍️ Product: https://experiences.myrealtrip.com/products/3446495
   5. 🛍️ Product: https://experiences.myrealtrip.com/products/3442583
   6. 🛍️ Product: https://experiences.myrealtrip.com/products/3887414
   7. 🛍️ Product: https://experiences.myrealtrip.com/products/3887426
   8. 🛍️ Product: https://experiences.myrealtrip.com/products/3446446
   9. 🛍️ Product: https://experiences.myrealtrip.com/products/3446422
  10. 🛍️ Produc

In [ ]:
## 0723. 제미나이 리팩토링 4그룹 코드 수정테스트 진행 해야 함###바로 아래 있는 셀은 검증완료 

# =============================================================================
# 🚀 그룹 4: 최종 데이터 수집 및 저장 (보수적 브라우저 재시작 포함)
# - 이 셀은 그룹 1에서 정의된 함수들을 '호출'하여 사용합니다.
# - 그룹 3에서 수집한 URL들의 상세 정보를 크롤링하고 최종 결과를 저장합니다.
# - 10-25개마다 랜덤하게 브라우저를 재시작하여 안전성을 극대화합니다.
# =============================================================================

import random  # 랜덤 재시작을 위한 import

# 🛡️ 안전한 브라우저 재시작 함수
def safe_browser_restart():
    """안전한 브라우저 재시작 with 3번 재시도"""
    global driver
    
    for attempt in range(3):  # 3번 시도
        try:
            print(f"🔄 브라우저 재시작 시도 {attempt+1}/3...")
            
            # 1단계: 안전한 종료
            if 'driver' in globals() and driver:
                driver.quit()
                driver = None
            
            # 2단계: 대기 및 정리
            wait_time = random.uniform(5, 10)
            print(f"⏰ {wait_time:.1f}초 대기 중...")
            time.sleep(wait_time)
            
            # 3단계: 새 브라우저 시작
            print("🚀 새 브라우저 시작 중...")
            driver = setup_driver()
            
            # 4단계: 동작 검증
            print("🔍 브라우저 동작 검증 중...")
            driver.get("https://www.myrealtrip.com/")
            time.sleep(2)
            
            print("✅ 브라우저 재시작 성공!")
            return True, "재시작 성공"
            
        except Exception as e:
            print(f"❌ 재시작 시도 {attempt+1} 실패: {type(e).__name__}: {e}")
            if attempt == 2:  # 마지막 시도
                print("🚨 브라우저 재시작 최종 실패!")
                return False, f"재시작 불가: {e}"
            print(f"🔄 {3-attempt-1}초 후 재시도...")
            time.sleep(3)  # 다음 시도 전 대기
    
    return False, "최종 실패"

print("🚀 그룹 4: 도시별 상세 정보 크롤링 시작!")
print("="*80)

# --- 1. 크롤링 준비 ---
# 그룹 1에서 설정된 도시 정보 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]
continent, country = get_city_info(city_name)

# 그룹 3에서 수집한 URL 중 설정된 개수만큼만 사용
urls_to_crawl = collected_urls[:CONFIG['MAX_PRODUCTS_PER_CITY']]
total_products = len(urls_to_crawl)

print(f"📊 그룹 1 설정값: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품")
print(f"🎯 실제 크롤링 대상: {total_products}개 상품")
print(f"🌍 위치: {continent} > {country} > {city_name}")
print(f"✈️ 공항 코드: {get_city_code(city_name)}")
print(f"📁 이미지 저장 기본 경로: myrealtripthumb_img/{continent}/{country}/{city_name}/")
print("="*80)

# --- 2. 메인 크롤링 루프 ---
all_results = []

# 🆕 보수적 브라우저 재시작 설정
restart_after = random.randint(10, 25)  # 보수적 재시작 간격
current_count = 0
print(f"🛡️ 보수적 모드: {restart_after}개 후 첫 번째 브라우저 재시작 예정")
print("="*80)

for product_index, product_url in enumerate(urls_to_crawl, 1):
    current_count += 1  # 🆕 카운트 증가
    
    # 🆕 보수적 재시작 체크 (크롤링 전에)
    if current_count > restart_after:
        print(f"\n🎲 {current_count-1}개 완료! 보수적 브라우저 재시작...")
        
        # 🛡️ 안전한 재시작 실행
        success, message = safe_browser_restart()
        
        if success:
            # 다음 재시작 간격 새로 설정
            restart_after = random.randint(10, 25)
            current_count = 1  # 카운트 리셋
            print(f"🎯 다음 재시작: {restart_after}개 후")
            print("="*50)
        else:
            print(f"🚨 브라우저 재시작 실패: {message}")
            print("⚠️ 현재 브라우저로 계속 진행하거나 수동으로 중단하세요.")
            # 재시작 카운트는 리셋하지 않고 계속 진행
            print("="*50)
    
    print_product_progress(product_index, total_products, f"상품 {product_index}")
    
    # 기본값 설정
    product_name, price_raw, price_clean, rating_raw, rating_clean = "정보 없음", "정보 없음", "정보 없음", "정보 없음", "정보 없음"
    review_count, language = "", ""
    img_info = {'status': '처리 안됨', 'filename': '', 'path': '', 'relative_path': '', 'size': 0}
    url_type = "Product" if "/products/" in product_url else "Offer"
    
    try:
        # 개별 상품 페이지로 이동
        print(f"    🔗 상품 {product_index} URL로 이동 중: {product_url}")
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
        
        # 그룹 1의 함수들을 사용하여 정보 수집
        product_name = get_product_name(driver, url_type)
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        rating_raw = get_rating(driver)
        rating_clean = clean_rating(rating_raw)
        review_count = get_review_count(driver)
        language = get_language(driver)
        
        # 그룹 1의 이미지 다운로드 함수 사용
        # (내부적으로 계층 구조 폴더 생성 로직이 포함되어 있음)
        img_info = download_image(driver, product_name, city_name, product_index)
        
        # 결과 저장
        result = {
            '번호': len(all_results) + 1, '대륙': continent, '국가': country, '도시': city_name,
            '공항코드': get_city_code(city_name), '상품번호': product_index, '상품타입': url_type,
            '상품명': product_name, '가격_원본': price_raw, '가격_정제': price_clean,
            '평점_원본': rating_raw, '평점_정제': rating_clean, '리뷰수': review_count, '언어': language,
            '이미지_파일명': img_info.get('filename', ''),
            '이미지_상대경로': img_info.get('relative_path', ''),
            '이미지_전체경로': img_info.get('path', ''),
            '이미지_상태': img_info.get('status', ''),
            '이미지_크기': img_info.get('size', 0),
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }
        all_results.append(result)
        print(f"    🎉 상품 {product_index} 정보 수집 완료: {product_name[:30]}...")

    except TimeoutException as e:
        print(f"    ⏰ 상품 {product_index} 타임아웃: 다음 상품으로 계속...")
        all_results.append({
            '번호': len(all_results) + 1, '대륙': continent, '국가': country, '도시': city_name,
            '공항코드': get_city_code(city_name), '상품번호': product_index, '상품타입': url_type,
            '상품명': f"타임아웃_{product_index}", '가격_원본': "수집실패", '가격_정제': "수집실패",
            '평점_원본': "수집실패", '평점_정제': "수집실패", '리뷰수': "", '언어': "",
            '이미지_파일명': "", '이미지_상대경로': "", '이미지_전체경로': "",
            '이미지_상태': "처리실패", '이미지_크기': 0, 'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'), '상태': 'TimeoutException'
        })
        continue
        
    except NoSuchElementException as e:
        print(f"    🔍 상품 {product_index} 요소 없음: 다음 상품으로 계속...")
        all_results.append({
            '번호': len(all_results) + 1, '대륙': continent, '국가': country, '도시': city_name,
            '공항코드': get_city_code(city_name), '상품번호': product_index, '상품타입': url_type,
            '상품명': f"요소없음_{product_index}", '가격_원본': "수집실패", '가격_정제': "수집실패",
            '평점_원본': "수집실패", '평점_정제': "수집실패", '리뷰수': "", '언어': "",
            '이미지_파일명': "", '이미지_상대경로': "", '이미지_전체경로': "",
            '이미지_상태': "처리실패", '이미지_크기': 0, 'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'), '상태': 'NoSuchElementException'
        })
        continue
        
    except WebDriverException as e:
        print(f"    🌐 상품 {product_index} 브라우저 오류: {type(e).__name__}")
        print(f"    🔧 브라우저 재시작을 시도합니다...")
        
        # 🛡️ 브라우저 오류 시 즉시 재시작 시도
        success, message = safe_browser_restart()
        if success:
            print(f"    ✅ 응급 재시작 성공! 다음 상품으로 계속...")
            # 재시작 카운트 리셋
            restart_after = random.randint(10, 25)
            current_count = 1
        else:
            print(f"    🚨 응급 재시작 실패: {message}")
            print(f"    ⛔ 크롤링을 중단합니다.")
            break
            
        # 실패 기록
        all_results.append({
            '번호': len(all_results) + 1, '대륙': continent, '국가': country, '도시': city_name,
            '공항코드': get_city_code(city_name), '상품번호': product_index, '상품타입': url_type,
            '상품명': f"브라우저오류_{product_index}", '가격_원본': "수집실패", '가격_정제': "수집실패",
            '평점_원본': "수집실패", '평점_정제': "수집실패", '리뷰수': "", '언어': "",
            '이미지_파일명': "", '이미지_상대경로': "", '이미지_전체경로': "",
            '이미지_상태': "처리실패", '이미지_크기': 0, 'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'), '상태': f'WebDriverException'
        })
        continue

# --- 3. 최종 결과 처리 및 저장 ---
print("\n🎉 크롤링 완료! 최종 결과를 저장합니다.")

if all_results:
    # 그룹 1의 저장 함수 호출
    saved_path = save_results(all_results)
    
    print(f"\n🏆 최종 저장 완료!")
    print(f"📁 CSV 파일: {saved_path}")
    print(f"📁 이미지 폴더: myrealtripthumb_img/{continent}/{country}/{city_name}/")
    print(f"📁 메타데이터: data_metadata.json")
    print(f"\n🛡️ 보수적 재시작 통계:")
    print(f"   📊 처리된 상품: {len(all_results)}개")
    print(f"   🔄 예상 재시작 횟수: 약 {len(all_results)//17}회")
else:
    print("⚠️ 수집된 데이터가 없어 저장할 파일이 없습니다.")

print("\n🎯 모든 작업이 완료되었습니다.")

In [ ]:
## 0723. 제미나이 리팩토링 4그룹 코드 수정. 검증 완료 0716.14:26

# =============================================================================
# 🚀 그룹 4: 최종 데이터 수집 및 저장
# - 이 셀은 그룹 1에서 정의된 함수들을 '호출'하여 사용합니다.
# - 그룹 3에서 수집한 URL들의 상세 정보를 크롤링하고 최종 결과를 저장합니다.
# =============================================================================

print("🚀 그룹 4: 도시별 상세 정보 크롤링 시작!")
print("="*80)

# --- 1. 크롤링 준비 ---
# 그룹 1에서 설정된 도시 정보 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]
continent, country = get_city_info(city_name)

# 그룹 3에서 수집한 URL 중 설정된 개수만큼만 사용
urls_to_crawl = collected_urls[:CONFIG['MAX_PRODUCTS_PER_CITY']]
total_products = len(urls_to_crawl)

print(f"📊 그룹 1 설정값: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품")
print(f"🎯 실제 크롤링 대상: {total_products}개 상품")
print(f"🌍 위치: {continent} > {country} > {city_name}")
print(f"✈️ 공항 코드: {get_city_code(city_name)}")
print(f"📁 이미지 저장 기본 경로: myrealtripthumb_img/{continent}/{country}/{city_name}/")
print("="*80)

# --- 2. 메인 크롤링 루프 ---
all_results = []

for product_index, product_url in enumerate(urls_to_crawl, 1):
    print_product_progress(product_index, total_products, f"상품 {product_index}")
    
    # 기본값 설정
    product_name, price_raw, price_clean, rating_raw, rating_clean = "정보 없음", "정보 없음", "정보 없음", "정보 없음", "정보 없음"
    review_count, language = "", ""
    img_info = {'status': '처리 안됨', 'filename': '', 'path': '', 'relative_path': '', 'size': 0}
    url_type = "Product" if "/products/" in product_url else "Offer"
    
    try:
        # 개별 상품 페이지로 이동
        print(f"    🔗 상품 {product_index} URL로 이동 중: {product_url}")
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
        
        # 그룹 1의 함수들을 사용하여 정보 수집
        product_name = get_product_name(driver, url_type)
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        rating_raw = get_rating(driver)
        rating_clean = clean_rating(rating_raw)
        review_count = get_review_count(driver)
        language = get_language(driver)
        
        # 그룹 1의 이미지 다운로드 함수 사용
        # (내부적으로 계층 구조 폴더 생성 로직이 포함되어 있음)
        img_info = download_image(driver, product_name, city_name, product_index)
        
        # 결과 저장
        result = {
            '번호': len(all_results) + 1, '대륙': continent, '국가': country, '도시': city_name,
            '공항코드': get_city_code(city_name), '상품번호': product_index, '상품타입': url_type,
            '상품명': product_name, '가격_원본': price_raw, '가격_정제': price_clean,
            '평점_원본': rating_raw, '평점_정제': rating_clean, '리뷰수': review_count, '언어': language,
            '이미지_파일명': img_info.get('filename', ''),
            '이미지_상대경로': img_info.get('relative_path', ''),
            '이미지_전체경로': img_info.get('path', ''),
            '이미지_상태': img_info.get('status', ''),
            '이미지_크기': img_info.get('size', 0),
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }
        all_results.append(result)
        print(f"    🎉 상품 {product_index} 정보 수집 완료: {product_name[:30]}...")

    except Exception as e:
        print(f"    ❌ 상품 {product_index} 처리 중 오류 발생: {type(e).__name__}: {e}")
        # 오류 발생 시에도 기본 정보 저장
        all_results.append({
            '번호': len(all_results) + 1, '대륙': continent, '국가': country, '도시': city_name,
            '공항코드': get_city_code(city_name), '상품번호': product_index, '상품타입': url_type,
            '상품명': f"오류_{product_index}", '가격_원본': "수집실패", '가격_정제': "수집실패",
            '평점_원본': "수집실패", '평점_정제': "수집실패", '리뷰수': "", '언어': "",
            '이미지_파일명': "", '이미지_상대경로': "", '이미지_전체경로': "",
            '이미지_상태': "처리실패", '이미지_크기': 0, 'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'), '상태': f'오류({type(e).__name__})'
        })
        continue

# --- 3. 최종 결과 처리 및 저장 ---
print("\n🎉 크롤링 완료! 최종 결과를 저장합니다.")

if all_results:
    # 그룹 1의 저장 함수 호출
    saved_path = save_results(all_results)
    
    print(f"\n🏆 최종 저장 완료!")
    print(f"📁 CSV 파일: {saved_path}")
    print(f"📁 이미지 폴더: myrealtripthumb_img/{continent}/{country}/{city_name}/")
    print(f"📁 메타데이터: data_metadata.json")
else:
    print("⚠️ 수집된 데이터가 없어 저장할 파일이 없습니다.")

print("\n🎯 모든 작업이 완료되었습니다.")


🚀 그룹 4: 도시별 상세 정보 크롤링 시작!
📊 그룹 1 설정값: 2개 상품
🎯 실제 크롤링 대상: 2개 상품
🌍 위치: 아시아 > 태국 > 푸켓
✈️ 공항 코드: HKT
📁 이미지 저장 기본 경로: myrealtripthumb_img/아시아/태국/푸켓/
    🎯 상품 진행률: [██████████░░░░░░░░░░] 50.0% (1/2)
    📦 현재 상품: 상품 1
    🔗 상품 1 URL로 이동 중: https://experiences.myrealtrip.com/products/3446452
  📊 Product 상품명 수집 중...
  💰 가격 정보 수집 중...
    ✅ 유효한 가격 발견: '52,000원~'
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 196개
  🌐 언어 정보 찾는 중...
    ✅ 언어 정보 발견: 한국어
  🖼️ 대표 상품 이미지 다운로드 중...
    ✅ 이미지 URL 발견: https://dry7pvlp22cox.cloudfront.net/mrt-images-pr...
    📁 계층 폴더 생성: myrealtripthumb_img\아시아\태국\푸켓
    ✅ 계층 구조 이미지 저장 완료! (632,419 bytes)
    📍 저장 위치: 아시아\태국\푸켓\HKT_001.jpg
    🎉 상품 1 정보 수집 완료: [푸켓/일일투어] 피피섬+마야베이+필레라군+ 몽키비치+...
    🎯 상품 진행률: [████████████████████] 100.0% (2/2)
    📦 현재 상품: 상품 2
    🔗 상품 2 URL로 이동 중: https://experiences.myrealtrip.com/products/3446456
  📊 Product 상품명 수집 중...
  💰 가격 정보 수집 중...
    ✅ 유효한 가격 발견: '22,000원~'
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 96개
  🌐 언어 정보 찾는 중...
    ✅ 언어 정보 발견: 한국